# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 2: Implementacion metodos de busqueda ciega para el Problema de Busqueda de Rutas en Mapas

La tarea de este laboratorio consiste en implementar y comparar métodos de busqueda ciega para buscar rutas en mapas.


Al final de este notebook debe responder algunas preguntas.

### Clase Mapa

Estructura para almacenar informacion de un mapa. Tiene dos atributos: <b>neighbors</b> (diccionario que contiene las ciudades vecinas de cada ciudad y las distancias para llegar a ellas) y <b>location</b>, diccionario con las coordenadas X,Y de cada ciudad.

In [1]:
class Mapa:
    def __init__(self, neighbors, location):
        self.neighbors = neighbors
        self.location = location

In [2]:
neighbors = {
 'A': [('Z',75), ('T',118), ('S',140)],
 'B': [('F',211), ('P',101), ('G',90), ('U',85)],
 'C': [('D',120), ('R',146), ('P',138)],
 'D': [('M',75), ('C',120)],
 'E': [('H',86)],
 'F': [('S',99), ('B',211)],
 'G': [('B',90)],
 'H': [('U',98), ('E',86)],
 'I': [('N',87), ('V',92)],
 'L': [('T',111), ('M',70)],
 'M': [('L',70), ('D',75)],
 'N': [('I',87)],
 'O': [('Z',71), ('S',151)],
 'P': [('R',97), ('C',138), ('B',101)],
 'R': [('S',80), ('C',146), ('P',97)],
 'S': [('A',140), ('O',151), ('F',99), ('R',80)],
 'T': [('A',118), ('L',111)],
 'U': [('B',85), ('V',142), ('H',98)],
 'V': [('U',142), ('I',92)],
 'Z': [('O',71), ('A',75)]}

location = {
 'A': (91, 492),
 'B': (400, 327),
 'C': (253, 288),
 'D': (165, 299),
 'E': (562, 293),
 'F': (305, 449),
 'G': (375, 270),
 'H': (534, 350),
 'I': (473, 506),
 'L': (165, 379),
 'M': (168, 339),
 'N': (406, 537),
 'O': (131, 571),
 'P': (320, 368),
 'R': (233, 410),
 'S': (207, 457),
 'T': (94, 410),
 'U': (456, 350),
 'V': (509, 444),
 'Z': (108, 531)}

romania = Mapa(neighbors, location)


### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [3]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

    def value(self, state):
        """En problemas de optimizacion, cada estado tiene un valor. Algoritmos
        como Hill-climbing intentan maximizar este valor."""
        raise NotImplementedError

###  <b> Clase MapSearchProblem </b>  
Esta es una subclase de SearchProblem donde se define concretamente el problema de busqueda en mapa. El constructor recibe el estado inicial, objetivo y un mapa. Se necesita completar Actions (acciones disponibles para un estado dado) y path_cost.

**Completar la función actions que retorna las acciones ejecutables desde una ciudad específica**

In [4]:
class MapSearchProblem(SearchProblem):
    def __init__(self, initial, goal, mapa):
        """El constructor recibe  el estado inicial, el estado objetivo y un mapa (de clase Mapa)"""
        self.initial = initial
        self.goal = goal
        self.map = mapa

    def actions(self, state):
        """Retorna las acciones ejecutables desde ciudad state.
        El resultado es una lista de strings tipo 'goCity'. 
        Por ejemplo, en el mapa de Romania, las acciones desde Arad serian:
         ['goZerind', 'goTimisoara', 'goSibiu']"""
        neighbors = []
        acciones = []
        tupla = ()
        neighbors = self.map.neighbors[state]
        # Escriba su solución acá
        for acc in range(len(neighbors)):
            acciones.append('go' + neighbors[acc][0])
        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde ciudad state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, en el mapa de Romania, el resultado de aplicar la accion 'goZerind' 
        desde el estado 'Arad' seria 'Zerind'"""  
        newState = action[2]
        return newState
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion debe ser
        extraido de self.map."""
        actionCost = 0;
        destStates = self.map.neighbors[state1] #estado destino, state2
        # Escriba su solución acá
        for acc in range(len(destStates)):
            if (destStates[acc][0] == state2):
                actionCost = destStates[acc][1]
                break
        return c + actionCost;

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

**Completar la función path que retorna la lista de nodos que va de la raíz a este nodo**

In [5]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        # Escriba su solución acá
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state

### <b> Definición de fronteras</b>
Se definen las clases correspondientes a colas FIFO y LIFO para usar como fronteras.

In [6]:
from collections import deque

class FIFO(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

class LIFO(deque):
    """Una cola Last-In-First-Out"""

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia FIFO o LIFO. Explora la frontera de acuerdo al algoritmo de búsqueda en árboles visto en clase



In [7]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    expanded_nodes = 0   # contador de nodos expandidos
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node, expanded_nodes
        explored.add(node.state)
        expanded_nodes = expanded_nodes + 1
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Probando los algoritmos de Busqueda</b> 
Ejecutar la búsqueda BFS, DFS y UCS aplicando la frontera correspondiente e imprimir los resultados. El output de la siguiente celda debe ser el siguiente:

Solucion obtenida con BFS: ['goS', 'goF', 'goB']. Nodos expandidos = 9.

Solucion obtenida con DFS: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 4. 

Solucion obtenida con UCS: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 12. 


In [8]:
p = MapSearchProblem('A', 'B', romania)   # problema de busqueda de ruta de Arad a Bucharest

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goS', 'goF', 'goB']. Nodos expandidos = 9
Solucion obtenida con DFS: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 4


## Preguntas

<b>0) Completar el código y ejecutar satisfactoriamente las pruebas</b> (3 pts)

<b>1) Probar BFS y DFS en los siguientes problemas (registre la ruta tomada y los nodos expandidos): </b> (4 pts)
* p = MapSearchProblem('A', 'D', romania)
* p = MapSearchProblem('F', 'Z', romania)
* p = MapSearchProblem('N', 'R', romania)
* p = MapSearchProblem('A', 'U', romania)
* p = MapSearchProblem('T', 'G', romania)

<b>2) Compare los nodos expandidos al buscar la ruta entre P y R, y explique la diferencia utilizando la teoría detrás de las búsquedas BFS y DFS </b> (4 pts)

<b>3) Justifique teóricamente la optimalidad (o no-optimalidad) de cada algoritmo</b> (4 pts)

<b>4) ¿En qué caso sugeriría usar cada algoritmo?</b> (3 pts)

<b>5) ¿Por qué se implementa una memoria de estados visitados en el algoritmo de búsqueda en grafo? </b> (2 pts)

In [9]:
#Pregunta 1 - Prueba 1
p = MapSearchProblem('A', 'D', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goT', 'goL', 'goM', 'goD']. Nodos expandidos = 12
Solucion obtenida con DFS: ['goS', 'goR', 'goC', 'goD']. Nodos expandidos = 13


In [10]:
#Pregunta 1 - Prueba 2
p = MapSearchProblem('F', 'Z', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goS', 'goA', 'goZ']. Nodos expandidos = 9
Solucion obtenida con DFS: ['goB', 'goP', 'goC', 'goD', 'goM', 'goL', 'goT', 'goA', 'goZ']. Nodos expandidos = 16


In [11]:
#Pregunta 1 - Prueba 3
p = MapSearchProblem('N', 'R', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goI', 'goV', 'goU', 'goB', 'goP', 'goR']. Nodos expandidos = 11
Solucion obtenida con DFS: ['goI', 'goV', 'goU', 'goB', 'goP', 'goR']. Nodos expandidos = 18


In [12]:
#Pregunta 1 - Prueba 4
p = MapSearchProblem('A', 'U', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goS', 'goF', 'goB', 'goU']. Nodos expandidos = 14
Solucion obtenida con DFS: ['goS', 'goR', 'goP', 'goB', 'goU']. Nodos expandidos = 5


In [13]:
#Pregunta 1 - Prueba 5
p = MapSearchProblem('T', 'G', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goA', 'goS', 'goF', 'goB', 'goG']. Nodos expandidos = 13
Solucion obtenida con DFS: ['goL', 'goM', 'goD', 'goC', 'goP', 'goB', 'goG']. Nodos expandidos = 13


In [14]:
#Pregunta 2 - Codigo para posterior analisis
p = MapSearchProblem('P', 'R', romania)

# Escriba su solución acá
node, num_exp_nodes = graph_search(p, FIFO())
print( 'Solucion obtenida con BFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = graph_search(p, LIFO())
print( 'Solucion obtenida con DFS: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

Solucion obtenida con BFS: ['goR']. Nodos expandidos = 1
Solucion obtenida con DFS: ['goR']. Nodos expandidos = 19


Pregunta 2 - Respuesta teorica: 
Como podemos observar, el metodo de busqueda que mas nodos expandio fue el DFS, esto debido a su naturaleza de expansion,
buscando siempre al nodo más profundo y en este caso pasando por alto una solución de pocos pasos. En cuanto al BFS, 
necesitaron de una expansión mucho menor, esto debido a la forma en la que lo hace, evaluando primero todos aquellos nodos
de cada nivel respectivamente. Es por ello que para este caso el numero de nodos expandidos con BFS sale mucho menor que con DFS.

Pregunta 3: 
Existen diferencias significativas en los costos a favor del BFS, esto se debe al recorrido que emplea cada método de búsqueda
ciega. Aunque el BFS requiera mayor tiempo de búsqueda, tenderá a encontrar la solución más sencilla(con menos pasos) dado que recorrera el mapa por niveles mientras que en el DFS será lo opuesto. En otras palabras el DFS requerira menor tiempo de busqueda pero los nodos expandidos son mayores por su naturaleza de expansion(profundidad).
Por otro lado, ya que el mapa no es muy grande podemos evitar el tiempo del analisis, siendo mas optimo el BFS para el caso propuesto.

Pregunta 4: 
Recomenaría usar DFS, siempre que se tenga un número muy amplio de nodos a visitar, aunque no garantice optimalidad al menos
será capaza de lidiar con tanta información en un tiempo prudente. Por otro lado, BFS es ideal para situaciones con un numero reducido de nodos, dado que el tiempo de respuesta para una solucion crece exponencialmente, pudiendo no terminar en un tiempo prudente, por otro lado, es más óptimo que el primero mencionado para el presente caso.
Ademas, no podemos olvidar que el BFS es mas complejo en memoria(se agota mas rapido) que el DFS.

Para este caso en particular usaria el BFS dado que no tenemos un numero muy grande de nodos, de tal forma que podemos recorrer el mapa por niveles. Esto lo podemos comprobar en las pruebas realizadas anteriormente ya que en su mayoria el numero de nodos expandidos para encontrar el objetivo por DFS es mayor que el de BFS.

Pregunta 5: 
Esto se da para evitar explorar nuevamente nodos que ya han sido explorados anteriormente y de esta forma hacer mas simple la solucion al problema. Por otro lado es importante resaltar que ya no se puede revertir la accion una vez hecha la mezcla y si realizamos un graph search, tal y como en el presente problema, y no tenemos una memoria de estados visitados en el algoritmo de búsqueda en grafo podriamos caer en un loop y de esta forma nunca encontrar el objetivo de la busqueda.